Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 5.23 ms, sys: 968 µs, total: 6.2 ms
Wall time: 13 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.1.68/10210/1 Dashboard: http://10.20.1.68:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 290 ms, sys: 41.6 ms, total: 332 ms
Wall time: 2.43 s


[1.4393986614224774,
 1.7151302761971874,
 1.729938817837224,
 1.1574608645420443,
 1.0934347268976032,
 1.8893027648825913,
 1.3447774587366061,
 1.1301094651757624,
 1.9952703937195093,
 1.3001294369412162,
 1.3463288855015354,
 1.9445705072108779,
 1.6967213637786216,
 1.0927660033973132,
 1.2900809565662574,
 1.5379164896084254,
 1.6067052348116744,
 1.0467980936154309,
 1.6612640120112432,
 1.490521284428195,
 1.1338662963174047,
 1.891701639555024,
 1.3810693741114974,
 1.5045965018984253,
 1.295918359948602,
 1.7284209303184448,
 1.2119618812759907,
 1.1840049535957013,
 1.0082630271932662,
 1.461329293723917,
 1.096654346747723,
 1.7496245623190558,
 1.4654605951943034,
 1.2710775451040033,
 1.5071520761147181,
 1.598676134405233,
 1.2693263518276625,
 1.4055018423820713,
 1.6557447823027847,
 1.3389412617164127,
 1.4752176204071925,
 1.7963982720487124,
 1.4132821656044872,
 1.5213207437441856,
 1.5086062574655692,
 1.4055752255720506,
 1.9577559655397878,
 1.2684966197309744,

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.1679556286312307,
 1.6212086598058935,
 1.8648661967500124,
 1.037671564483876,
 1.492587847380714,
 1.0559105008482172,
 1.3710604856368858,
 1.9701447752252195,
 1.0578106706748,
 1.3019860658174411,
 1.7832635311414484,
 1.1067786027209787,
 1.4037800123029687,
 1.9039185185432217,
 1.0622227892067138,
 1.85427877492088,
 1.8978814543000624,
 1.2416178628669425,
 1.1882995928379598,
 1.6127744221174616,
 1.2465409944096724,
 1.3969830563608006,
 1.3790532957562818,
 1.2013505794335484,
 1.9727657194694102,
 1.6186220194064618,
 1.8440970012949807,
 1.4882903856980811,
 1.4295967226452486,
 1.8631999197874345,
 1.4978793761320002,
 1.3077438139005348,
 1.4998315426549702,
 1.0226411565842506,
 1.1313387682155867,
 1.016743366756403,
 1.561772868040392,
 1.8395604178886895,
 1.4298144060766997,
 1.435019054662511,
 1.206226761376631,
 1.3431745065507688,
 1.3952805408691777,
 1.5716226286155695,
 1.4690554044998825,
 1.14225576442364,
 1.7188212714195106,
 1.2167019092937776,
 1.98

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 237 ms, sys: 22.3 ms, total: 260 ms
Wall time: 2.46 s
